In [1]:
# Configure Drive or Jupyter notebook -- only runs when first loaded
if "CONFIG_DONE" not in globals():
    # Need to mount drive and clone repo to access data and functions
    try:
        from google.colab import drive  # type: ignore

        IN_COLAB = True

        # clone repo
        !git clone https://github.com/doctorsmylie/mtg-draft-agent
        %cd mtg-draft-agent

    except ModuleNotFoundError:
        IN_COLAB = False

    # Finish configuration -- also configures notebook outside of Colab
    %run "project_path.ipynb"
else:
    print("Config done already")

Starting config...
Running in Colab? No

Configuring Jupyter Notebook...
cd to the repo's base directory...

BASE_PATH =  /home/mrgomez/Documents/Erdos/DL_2025/mtg-draft-agent
DATA_FOLDER = /home/mrgomez/Documents/Erdos/DL_2025/mtg-draft-agent/MTGdraft
BASE_PATH == os.getcwd(): True

Configuration done


Testing queries to the Scryfall API to obtain card text and attributes.

In [ ]:
import requests
from time import time, sleep

from functions.scraping import *

# Testing syntax

In [3]:
# Scryfall asks us to specify the user and the format of the data
headers = {
    "User-Agent": "2025-dl-mtg-draft",  # Name of our app
    "Accept": "application/json",  # Return format in json
}

In [4]:
# Ping the homepage of Scryfall
scryfall_url_home = "https://api.scryfall.com"
response = requests.get(scryfall_url_home, headers=headers)
response.json()

{'object': 'error',
 'code': 'bad_request',
 'status': 400,
 'details': 'This is the root of the Scryfall API and no data is returned at this path. For more information about the methods and objects this API publishes, please see https://scryfall.com/docs/api'}

# Search for a specific expansion

In [5]:
# Search for cards in a specific expansion
expansion = "dsk"
cards, responses, responses_json = get_expansion_from_scryfall(
    expansion, delay=50e-3, return_json=True
)

In [6]:
# Debugging: checking the request URLs and number of cards
for i in range(len(responses)):
    print("URL:", responses[i].url)
    print(responses_json[i].keys())
    print("Has more:", responses_json[i]["has_more"])
    print("Num. cards", responses_json[i]["total_cards"])
    print()

URL: https://api.scryfall.com/cards/search?q=set%3Adsk&page=1
dict_keys(['object', 'total_cards', 'has_more', 'next_page', 'data'])
Has more: True
Num. cards 277

URL: https://api.scryfall.com/cards/search?q=set%3Adsk&page=2
dict_keys(['object', 'total_cards', 'has_more', 'data'])
Has more: False
Num. cards 277



In [7]:
# Show the attributes of the cards (specifically, one card)
# idc = id card
idc = 20

keys = list(responses_json[0]["data"][idc].keys())
keys.sort()
for k in keys:
    print(k)

arena_id
artist
artist_ids
booster
border_color
card_back_id
card_faces
cardmarket_id
cmc
collector_number
color_identity
colors
digital
edhrec_rank
finishes
foil
frame
full_art
game_changer
games
highres_image
id
illustration_id
image_status
image_uris
keywords
lang
layout
legalities
mana_cost
mtgo_id
multiverse_ids
name
nonfoil
object
oracle_id
oversized
penny_rank
preview
prices
prints_search_uri
promo
purchase_uris
rarity
related_uris
released_at
reprint
reserved
rulings_uri
scryfall_set_uri
scryfall_uri
set
set_id
set_name
set_search_uri
set_type
set_uri
story_spotlight
tcgplayer_id
textless
type_line
uri
variation


## Show some cards in the set

In [8]:
cards = responses_json[0]["data"]

for idc in range(10):
    card = cards[idc]

    print_card_attributes(card)

--------------- 
Name: Abandoned Campground
Rarity: common
Type: Land

This land enters tapped unless a player has 13 or less life.
{T}: Add {W} or {U}.
--------------- 

--------------- 
Name: Abhorrent Oculus
Rarity: mythic
Cost: {2}{U}
Type: Creature — Eye
P/T: 5/5

As an additional cost to cast this spell, exile six cards from your graveyard.
Flying
At the beginning of each opponent's upkeep, manifest dread. (Look at the top two cards of your library. Put one onto the battlefield face down as a 2/2 creature and the other into your graveyard. Turn it face up any time for its mana cost if it's a creature card.)
--------------- 

--------------- 
Name: Acrobatic Cheerleader
Rarity: common
Cost: {1}{W}
Type: Creature — Human Survivor
P/T: 2/2

Survival — At the beginning of your second main phase, if this creature is tapped, put a flying counter on it. This ability triggers only once.
--------------- 

--------------- 
Name: A-Leyline of Resonance
Rarity: rare
Cost: {2}{R}{R}
Type: Enc

### Test edge cases of display function

In [9]:
# Show Planeswalkers
for card in cards:
    if "Planeswalker" in card["type_line"]:
        print_card_attributes(card)

# Show two-sided cards
for card in cards:
    if is_two_sided(card):
        print_card_attributes(card)

--------------- 
Name: Kaito, Bane of Nightmares
Rarity: mythic
Cost: {2}{U}{B}
Type: Legendary Planeswalker — Kaito
L: 4

Ninjutsu {1}{U}{B} ({1}{U}{B}, Return an unblocked attacker you control to hand: Put this card onto the battlefield from your hand tapped and attacking.)
During your turn, as long as Kaito has one or more loyalty counters on him, he's a 3/4 Ninja creature and has hexproof.
+1: You get an emblem with "Ninjas you control get +1/+1."
0: Surveil 2. Then draw a card for each opponent who lost life this turn.
−2: Tap target creature. Put two stun counters on it.
--------------- 

--------------- 
Bottomless Pool // Locker Room
Rarity: uncommon
// --- //
Name: Bottomless Pool
Cost: {U}
Type: Enchantment — Room

When you unlock this door, return up to one target creature to its owner's hand.
(You may cast either half. That door unlocks on the battlefield. As a sorcery, you may pay the mana cost of a locked door to unlock it.)

// --- //
Name: Locker Room
Cost: {4}{U}
Type: